# Import packages

In [5]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
import torch

from Gaen_packages.Gaen_packages import ImageTreatment_and_interfaces as ii


In [6]:
np.__version__

'1.25.2'

# Load images

In [4]:
# file_path = 'auxiliar_data\\atoms.jpg'
file_path = r'C:\Users\ipinto\OneDrive - INSTITUT CATALA DE NANOCIENCIA I NANOTECNOLOGIA\PROGRAMMING\Proyectos\Hackathon-2024-GAeN\Ivan\imagenes_M\labels_con_ruido\label_0_n0.png'
file_path2 = r'C:\Users\ipinto\OneDrive - INSTITUT CATALA DE NANOCIENCIA I NANOTECNOLOGIA\PROGRAMMING\Proyectos\Hackathon-2024-GAeN\Ivan\imagenes_M\imagenes_con_ruido\imagen_0_n0.png'

image = np.array(Image.open(file_path2))
labels = np.array(Image.open(file_path))
if len(image.shape) == 3:
    image = image[:,:,0]

# file_path = 'S:\Ivan Pinto\Helena\\0028 - AlOx-BiVO4 HAADF-DF 5.20 Mx HAADF.dm4'
# signal = hs.load(file_path)
# image = signal.data
%matplotlib qt
ii.multiplot(image, labels)

In [8]:
model = torch.load('model_tecnai.pt')


ModuleNotFoundError: No module named 'numpy._core'